In [337]:
import pandas as pd
from PIL import Image
import cv2
import numpy as np
import math

In [449]:
original_images = context.catalog.load("express_expense")

[05/28/23 22:04:27] INFO     Loading data from 'express_expense' (PartitionedDataSet)...        ]8;id=280959;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=581605;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [491]:
points_auto = context.catalog.load("homography_points_segmentation_batched_hough")

[05/28/23 22:16:38] INFO     Loading data from 'homography_points_segmentation_batched_hough'   ]8;id=535718;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=615424;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (CSVDataSet)...                                                                       

In [492]:
points_auto

,index,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y,mask_index
0,batched_1000mask0,690,958,658,181,114,958,173,198,0
1,batched_1000mask1,665,178,108,997,172,195,715,997,1
2,batched_1001mask2,76,952,545,74,64,45,569,905,2
3,batched_1002mask1,362,997,79,28,23,997,446,113,1
4,batched_1002mask2,371,997,26,997,456,997,524,997,2
...,...,...,...,...,...,...,...,...,...,...
386,batched_1197mask2,288,96,293,93,288,93,293,96,2
387,batched_1198mask2,233,447,0,447,215,447,227,447,2
388,batched_1199mask0,304,345,58,162,290,160,39,345,0
389,batched_1199mask1,77,0,37,345,310,345,277,0,1


In [493]:
points_auto = points_auto.rename(columns={'index': 'Image'})
#points_auto["Image"] = points_auto['Image'].str.replace("-receipt.jpg", "").astype('uint32')
#points_auto["Image"] = points_auto['Image'].str.replace("batched_", "").astype('uint32')
points_auto["Image"] = points_auto['Image'].str.replace("batched_", "").str.replace("mask.*", "", regex=True).astype('uint32')
points_auto = points_auto.set_index("Image")

In [494]:
points_auto

,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y,mask_index
Image,,,,,,,,,
1000,690,958,658,181,114,958,173,198,0
1000,665,178,108,997,172,195,715,997,1
1001,76,952,545,74,64,45,569,905,2
1002,362,997,79,28,23,997,446,113,1
1002,371,997,26,997,456,997,524,997,2
...,...,...,...,...,...,...,...,...,...
1197,288,96,293,93,288,93,293,96,2
1198,233,447,0,447,215,447,227,447,2
1199,304,345,58,162,290,160,39,345,0


In [495]:
points_manual = pd.read_csv("../data/01_raw/homography.csv", sep=";")

In [496]:
points_manual['P1X'] = points_manual['P1'].str.split(',').apply(lambda x: x[0]).astype('uint32')

In [497]:
points_manual['P1Y'] = points_manual['P1'].str.split(',').apply(lambda x: x[1]).astype('uint32')

In [498]:
points_manual['P2X'] = points_manual['P2'].str.split(',').apply(lambda x: x[0]).astype('uint32')

In [499]:
points_manual['P2Y'] = points_manual['P2'].str.split(',').apply(lambda x: x[1]).astype('uint32')

In [500]:
# o P3 está invertido com o P4 na detecção manual, não sei por qual motivo

In [501]:
points_manual['P3X'] = points_manual['P4'].str.split(',').apply(lambda x: x[0]).astype('uint32')

In [502]:
points_manual['P3Y'] = points_manual['P4'].str.split(',').apply(lambda x: x[1]).astype('uint32')

In [503]:
points_manual['P4X'] = points_manual['P3'].str.split(',').apply(lambda x: x[0]).astype('uint32')

In [504]:
points_manual['P4Y'] = points_manual['P3'].str.split(',').apply(lambda x: x[1]).astype('uint32')

In [505]:
points_manual = points_manual.drop(columns=['P1', 'P2', 'P3', 'P4'])

In [506]:
points_manual = points_manual.set_index("Image")

### Apenas 73% das imagens foram reconhecidas automaticamente

In [507]:
missing = set(points_manual.index) - set(points_auto.index)
missing

{1006,
 1034,
 1041,
 1044,
 1053,
 1062,
 1063,
 1068,
 1073,
 1087,
 1114,
 1122,
 1145,
 1146,
 1147,
 1177,
 1182,
 1183,
 1188,
 1194}

In [508]:
points_manual = points_manual.loc[points_auto.index]

In [509]:
points_manual

,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y
Image,,,,,,,,
1000,182,199,661,193,699,929,119,914
1000,182,199,661,193,699,929,119,914
1001,83,77,545,90,576,788,72,824
1002,76,110,413,179,355,998,1,981
1002,76,110,413,179,355,998,1,981
...,...,...,...,...,...,...,...,...
1197,25,116,287,101,308,445,19,437
1198,2,1,218,5,231,446,1,445
1199,69,1,278,3,309,344,35,341


**Problemas em calcular a diferença entre os pontos manuais e os automáticos**
1. A distância entre dois pontos deve ser considerada a mais curta, do teorema de Pitágoras?
2. As imagens têm resoluções diferentes, o que impacta nessa conta das métricas

Interessante mencionar no artigo. Dito isso, vamos aos cálculos:

1. Diferença entre os pontos manuais e os pontos automáticos (Pitágoras)

In [510]:
dp1 = np.sqrt(pow(points_auto['P1X'] - points_manual['P1X'], 2) + pow(points_auto['P1Y'] - points_manual['P1Y'], 2))
dp2 = np.sqrt(pow(points_auto['P2X'] - points_manual['P2X'], 2) + pow(points_auto['P2Y'] - points_manual['P2Y'], 2))
dp3 = np.sqrt(pow(points_auto['P3X'] - points_manual['P3X'], 2) + pow(points_auto['P3Y'] - points_manual['P3Y'], 2))
dp4 = np.sqrt(pow(points_auto['P4X'] - points_manual['P4X'], 2) + pow(points_auto['P4Y'] - points_manual['P4Y'], 2))

In [511]:
dt = np.array(np.ravel(np.concatenate((dp1, dp2, dp3, dp4)).reshape(-1)))

In [512]:
dt.max()

4369.799194471068

In [513]:
dt.min()

1.0

In [514]:
dt.mean()

582.3804521308189

In [515]:
np.median(dt)

578.0471420663318

In [516]:
dt.std()

422.34977053266834

In [517]:
np.argmax(dt)

689